<a href="https://www.kaggle.com/code/amirmotefaker/sales-product-analysis?scriptVersionId=130327936" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import Libraries

In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import warnings

from matplotlib import pylab as plt  # PyLab is a procedural interface to the Matplotlib object-oriented plotting library.

from statsmodels.graphics.gofplots import qqplot  # Can take arguments specifying the parameters for dist or fit them automatically. 

from IPython.core.interactiveshell import InteractiveShell  # An enhanced, interactive shell for Python.

# Merge the 12 months of sales data into a single CSV file

In [ ]:
# let's make a list compreension for all the data in the folder
files = [file for file in os.listdir('/kaggle/input/sales-product-data')] 

# let's make a pandas DataFrame
all_months_data = pd.DataFrame()

# makes a loop for concat the data
for file in files:
    data = pd.read_csv("/kaggle/input/sales-product-data/" + file)
    all_months_data = pd.concat([all_months_data, data])

# export all data to csv    
all_months_data.to_csv("/kaggle/working/all_data.csv", index=False)

# Read in updated DataFrame

In [ ]:
# read data
sales_data = pd.read_csv('all_data.csv') 

# show data
sales_data 

In [ ]:
"Head"
# Checking the first 5 rows of data
sales_data.head() 

"Tail"
# Checking the last 5 rows of data
sales_data.tail() 

# Data Preprocessing

In [ ]:
# getting the information 
sales_data.info()

# Uniqueness Categorical Variables

In [ ]:
# getting the Uniqueness catrgorical variable
categorical = sales_data.select_dtypes(['category', 'object']).columns 
for col in categorical:
    print('{} : {} unique value(s)'.format(col, sales_data[col].nunique()))

# How many missing data points do we have?

In [ ]:
# get the number of missing data points per column
missing_values_count = sales_data.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

# What percentage of the values in our dataset were missing to give us a better sense of the scale of this problem

In [ ]:
# how many total missing values do we have?
total_cells = np.product(sales_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing / total_cells) * 100
print(f"{percent_missing:.2f}%")

# Clean up the Data

In [ ]:
# let's drop the rows of NaN data!
sales_data = sales_data.dropna(how='all')

# okay, let's check it again!
"NaN Value:"
sales_data[sales_data.isna().any(axis=1)]

# future warning! ValueError: invalid literal for int() with base 10: 'Or'
"Clean Future Warnings:"
sales_data = sales_data[sales_data['Order Date'].str[0:2] != 'Or']
sales_data

# Convert Quantity Ordered column and Price Each column

In [ ]:
# convert the data
sales_data['Quantity Ordered'], sales_data['Price Each'] = sales_data['Quantity Ordered'].astype('int64'), sales_data['Price Each'].astype('float')

# check it 
sales_data.info()

# Convert Order Date column

In [ ]:
# convert it using to_datetime() funct
sales_data['Order Date'] = pd.to_datetime(sales_data['Order Date'])

# check data
sales_data

### Recap Data:

- We have total  186850  records and  6  columns cateogircal type

- The total of missing value that we have is  0.29167 %

- Order ID :  178438  unique value(s)

- Product :  20  unique value(s)

- Quantity Ordered :  10  unique value(s)

- Price Each :  24  unique value(s)

- Order Date :  142396  unique value(s)

- Purchase Address :  140788  unique value(s)

# Data Preparation

## Add Month, Hour, Minute, Sales, Cities Column

In [ ]:
def augment_data(data):
    
    """
    Adding new features to
    our data, adding Month Data,
    Hour Data, Minute Data, Sales Data,
    and Cities Column
    
    Returning:
        data with new features
    """
    
    # funtction to get the city in the data
    def get_city(address):
        return address.split(',')[1]
    
    # funtction to get the state in the data
    def get_state(address):
        return address.split(',')[2].split(' ')[1]

    # let's get the year data in order date column
    data['Year'] = data['Order Date'].dt.year
    
    # let's get the month data in order date column
    data['Month'] = data['Order Date'].dt.month
    
    # let's get the houe data in order date column
    data['Hour'] = data['Order Date'].dt.hour 
    
    # let's get the minute data in order date column
    data['Minute'] = data['Order Date'].dt.minute 
    
    # let's make the sales column by multiplying the quantity ordered colum with price each column
    data['Sales'] = data['Quantity Ordered'] * data['Price Each'] 
    
    # let's get the cities data in order date column
    data['Cities'] = data['Purchase Address'].apply(lambda x: f"{get_city(x)} ({get_state(x)})") 
    
    return data # returning data

# and see it
sales_data = augment_data(sales_data)
sales_data.head()

# Data Analysis

In [ ]:
# set the seaborn style
sns.set_style("whitegrid") 

# let's make a correlation matrix for `cop_data`
plt.figure(figsize=(24, 18)) # figure the size
sns.heatmap(sales_data.corr(), annot=True) # create a heatmap
plt.title("Sales Data Correlation", weight="bold", fontsize=35, pad=30) # title
plt.xticks(weight="bold", fontsize=15) # x-ticks
plt.yticks(weight="bold", fontsize=15); # y-ticks

In [ ]:
# Let's see the correlation from `sales_data`
(sales_data.corr()['Sales'] # transform it into data corr
           .sort_values(ascending=False) # sort values
           .to_frame() # change it into data frame
           .T) # transpose it

In [ ]:
# statistical measure of sales data without object type of data
sales_data_numeric = sales_data.describe(include=[np.number]) 
"Statistical Measure of Sales Data in Numeric Data"
sales_data_numeric

In [ ]:
# statistical measure of sales data without numeric type of data
sales_data_object = sales_data.describe(exclude=[np.number])
"Statistical Measure of Sales Data in Object / Str Data"
sales_data_object

# Univariate Analysis

In [ ]:
# checking and visualizing the type of distribution of a feature column
def univariate_analysis(data, color, title1, title2):
    
    """
    Showing visualization of univariate
    analysis with displot and qqplot
    visualization from seaborn and statsmodel
    library.
    
    Parameters
    ----------
    data : DataFrame, array, or list of arrays, optional
        Dataset for plotting. If ``x`` and ``y`` are absent, this is
        interpreted as wide-form. Otherwise it is expected to be long-form. 
    title1: The title of the visualization, title1 for displot visualization
        And title2 for quantile plot from statsmodel.
    title2: The title of the visualization, title1 for displot visualization
        And title2 for quantile plot from statsmodel.
        
    Returns
    -------
    fig : matplotlib figure
        Returns the Figure object with the plot drawn onto it.
    """
    
    fig, (ax1, ax2) = plt.subplots( # subplots
        ncols=2, # num of cols
        nrows=1, # num of rows
        figsize=(20, 6) # set the width and high
    )

    sns.distplot( # create a distplot visualization
        data, # data
        ax=ax1, # axes 1
        kde=True, # kde
        color=color # color
    )
    
    ax1.set_title( # set the title 1
        title1, 
        weight="bold", # weight
        fontsize=25, # font-size
        pad=30 # padding
    ) 
    
    qqplot( # qqplot (quantile plot)
        data, # data
        ax=ax2, # axes 2
        line='s' # line 
    )
    
    ax2.set_title( # set the title 2
        title2, 
        weight="bold", # weight
        fontsize=25, # font-size
        pad=30 # padding
    )
    
    return fig # returning the figure

In [ ]:
# Quantity Ordered Data
univariate_analysis( # call the function
    data=sales_data['Quantity Ordered'], # put the data
    color='red', # pick the color
    title1='Quantity Ordered Data Distribution', # title1
    title2='Quantile Plot' # title2
);

# Price Each Data

In [ ]:
# Price Each Data
univariate_analysis( # call the function
    data=sales_data['Price Each'], # put the data
    color='blue', # pick the color 
    title1='Price Each Data Distribution', # title1 
    title2='Quantile Plot' # title2
);

# Sales Data

In [ ]:
# Sales Data
univariate_analysis( # call the function
    data=sales_data['Sales'], # put the data 
    color='black', # pick the color
    title1='Sales Data Distribution', # title1 
    title2='Quantile Plot' # title2
);

# Checking skewness value

In [ ]:
# checking skewness value
# if value lies between -0.5 to 0.5  then it is normal otherwise skewed
skew_value = sales_data.skew().sort_values(ascending=False)
skew_value

# What was the best Year for sales? How much was earned that Year?

In [ ]:
# let's plot it
plt.figure(figsize=(24, 10)) # figuring the size

# makes count plot 
sns.countplot(
    x="Year", 
    data=sales_data
)
plt.title( # title
    "Year Sales and Much Earned in that Year", 
    fontname="monospace", # font-name
    weight="bold", # weiqht
    fontsize=35, # font-size
    pad=30 # padding
)
plt.xlabel( # x-label
    "Years", 
    weight="bold", # weight
    color="purple", # color
    fontsize=25, # font-size
    loc="center" # location
)
plt.xticks( # x-ticks
    weight="bold", # weight
    fontsize=15 # font-size
)
plt.ylabel( # y-label
    "Sales in USD ($)", 
    weight="bold", # weight
    color="green", # color
    fontsize=20 # font-size
)
plt.yticks( # y-ticks
    weight="bold", # weight 
    fontsize=15 # font-size
);

# What was the best month for sales? How much was earned that month?

In [ ]:
# group the Month cols
sum_of_month_and_earned = sales_data.groupby('Month').sum().astype('int')

# let's plot it
plt.figure(figsize=(24, 14)) # figuring the size

# makes bar plot 
sns.barplot( # barplot
    x=sum_of_month_and_earned.index, # x-axis
    y=sum_of_month_and_earned["Sales"], # y-axis
    data=sum_of_month_and_earned, # data
    palette="deep" # palette
)
plt.title( # title
    "Month Sales and Much Earned in that Months", 
    fontname="monospace", # font-name
    weight="bold", # weight
    fontsize=35, # font-size
    pad=30 # padding
)
plt.xlabel( # x-label
    "Months", 
    weight="bold", # weight
    color="purple", # color
    fontsize=25, # font-size
    loc="center" # location
)
plt.xticks( # x-ticks
    weight="bold", # weight
    fontsize=15 # font-size
)
plt.ylabel( # y-label
    "Sales in USD ($)", 
    weight="bold", # weight
    color="green", # color
    fontsize=20 # font-size
)
plt.yticks( # y-ticks
    weight="bold", # weight 
    fontsize=15 # font-size
);

# What City had the highest number of sales?

In [ ]:
# group of the highest number of sales in city
highest_number_of_sales = sales_data.groupby('Cities').sum().astype('int')

# let's plot it
plt.figure(figsize=(24, 14)) # figuring the size

# makes bar plot 
sns.barplot( # barplot
    x=highest_number_of_sales.index, # x-axis
    y=highest_number_of_sales["Sales"], # y-axis
    data=highest_number_of_sales, # data
    palette="deep" # palette
)
plt.title( # title
    "City with the Highest number of Sales", 
    fontname="monospace", # font-name
    weight="bold", # weight
    fontsize=35, # font-size
    pad=30 # padding
)
plt.xlabel( # x-label
    "Cities", 
    weight="bold", # weight
    color="purple", # color
    fontsize=25, # font-size
    loc="center" # location
)
plt.xticks( # x-ticks
    weight="bold", # weight
    fontsize=15, # font-size
    rotation=10
)
plt.ylabel( # y-label
    "Sales in USD ($)", 
    weight="bold", # weight
    color="green", # color
    fontsize=20 # font-size
)
plt.yticks( # y-ticks
    weight="bold", # weight 
    fontsize=15 # font-size
);

# What time should we display adverstisement to maximize likelihood of customer's buying product?

In [ ]:
# let's prepare the value for the x-axis
hours = [hour for hour, df in sales_data.groupby('Hour')]

#let's plot it
plt.figure(figsize=(24, 10)) # figuring the size

# makes bar plot 
plt.plot( # plot
    hours, # x-axis
    sales_data.groupby(['Hour']).count() # data
)

# let's add grid
plt.grid(True)
plt.title( # title
    "What time should we display adverstisement to maximize likelihood of customer's buying product?", 
    weight="bold", # weight
    fontsize=35, # font-size
    pad=30
)
plt.xlabel( # x-label
    "Hours", 
    weight="bold", # weight
    color="purple", # color
    fontsize=25, # font-size
    loc="center" # location
)
plt.xticks( # x-ticks
    ticks=hours, # labels
    weight="bold", # weight
    fontsize=15 # font-size
)
plt.ylabel( # y-label
    "Number of Orders", 
    weight="bold", # weight
    color="black", # color
    fontsize=20 # font-size
)
plt.yticks( # y-ticks
    weight="bold", # weight 
    fontsize=15 # font-size
);

# What products are most often sold together?

In [ ]:
from itertools import combinations
from collections import Counter

# drop it using duplicated() funct
data = sales_data[sales_data['Order ID'].duplicated(keep=False)]

# create a new column
data['Grouped'] = sales_data.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))

# let's make a new variable
data = data[['Order ID', 'Grouped']].drop_duplicates()

# create a new variable for Counter
count = Counter()

# make a for loop
for row in data['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))

# and make another for loop    
for key, value in count.most_common(10):
    print(key, value)

# What product sold the most? Why do you think it sold the most?

In [ ]:
# lets prepare the varables for the plotting
# group the product
product_group = sales_data.groupby('Product')
quantity_ordered = product_group.sum()['Quantity Ordered']
prices = sales_data.groupby('Product').mean()['Price Each']
products = [product for product, df in product_group]
"""Visualization"""

# let's make a subplots
fig, ax1 = plt.subplots(figsize=(24, 14))
ax2 = ax1.twinx()

# AXES 1
ax1.bar(products, quantity_ordered)
ax1.set_title( # title
    "Product with the Price",
    weight="bold", # weight
    fontname="monospace", # font-name
    fontsize=35, # font-size
    pad=30 # padding
)
ax1.set_xlabel( # x-label
    "Product Names",
    weight="bold", # weight
    fontsize=25, # font-size
    color="black" # color
)
ax1.set_ylabel( # y-label
    "Quantity Ordered", 
    color="blue", # color
    fontsize=20, # font-size
    weight="bold" # weight
)
ax1.set_xticklabels( # x-ticks
    products, # ticks
    rotation="vertical", # rotation 
    weight="bold" # weight
)

# AXES 2
ax2.plot( # plot
    products, # x-axis
    prices, # y-axis
    "r-"
)
ax2.set_ylabel( # y-label
    "Price in USD ($)", 
    color="green", # color
    fontsize=20, # font-size
    weight="bold" # weight
);

In [ ]:
sales_data.Product.value_counts().to_frame().T

# How much probability for next people will order USB-C Charging Cable?

In [ ]:
def statistical_probability(frequency, total_frequency):
    return frequency / total_frequency

product = sales_data.Product.value_counts().sum()

In [ ]:
usb_charging = sales_data[sales_data.Product == 'USB-C Charging Cable'].value_counts().sum()

# Calculating USB-C Charging Probability
P_USB = statistical_probability(usb_charging, product)
Pprime_USB = 1 - P_USB
print('Probability for next people will order USB-C Charging Cable: %.2f%%' % P_USB)
print('Probability for next people will not order USB-C Charging Cable: %.2f%%' % Pprime_USB)

# How much probability for next people will order iPhone?

In [ ]:
iphone = sales_data[sales_data.Product == 'iPhone'].value_counts().sum()

# Calculating iPhone Probability
P_iphone = statistical_probability(iphone, product)
Pprime_iphone = 1 - P_iphone
print('Probability for next people will order iPhone: %.2f%%' % P_iphone)
print('Probability for next people will not order iPhone: %.2f%%' % Pprime_iphone)

# How much probability for next people will order Google Phone?

In [ ]:
google_phone = sales_data[sales_data.Product == 'Google Phone'].value_counts().sum()

# Calculating Google Phone Probability
P_google_phone = statistical_probability(google_phone, product)
Pprime_google_phone = 1 - P_google_phone
print('Probability for next people will order Google Phone: %.2f%%' % P_google_phone)
print('Probability for next people will not order Google Phone: %.2f%%' % Pprime_google_phone)